# 匯入套件

In [1]:
import requests
import pandas as pd
from lxml import etree
from bs4 import BeautifulSoup
import json
import ast
import re
import time

# 網頁檢索

In [2]:
res_each = requests.get('https://www.beautyclinic.com.tw/page/doctors')
raw_data_each = etree.HTML(res_each.content.decode('utf-8'))
temp = raw_data_each.xpath("//*[@id='content']/script/text()")

# 取得診所資訊

In [3]:
clinicname=[]
detect=raw_data_each.xpath("//div[contains(@class,'col_half')]//li[position()>=0 and position()<=4]/text()")
for i in range(len(detect)):
    clinicname.append(detect[i].split(' /')[0].strip())
    okclinicname = [t for t in clinicname if t != '']
# 抓取各家診所資訊(地址﹑電話﹑網址﹑城市)
otherinfo = {
    okclinicname[0]:{
        "address":raw_data_each.xpath("//div[contains(@class,'col_half')]//li[1]/div/text()[2]"),
        "tel":raw_data_each.xpath("//div[contains(@class,'col_half')]//li[1]/div/text()[1]"),
        "store_website":raw_data_each.xpath("//*[@id='main-menu']/li[10]/ul/li[1]/a/@href"),
        "city":okclinicname[0]
    },okclinicname[1]:{
        "address":raw_data_each.xpath("//div[contains(@class,'col_half')]//li[2]/div/text()[2]"),
        "tel":raw_data_each.xpath("//div[contains(@class,'col_half')]//li[2]/div/text()[1]"),
        "store_website":raw_data_each.xpath("//*[@id='main-menu']/li[10]/ul/li[2]/a/@href"),
        "city":okclinicname[1]
    },okclinicname[2]:{
        "address":raw_data_each.xpath("//div[contains(@class,'col_half')]//li[3]/div/text()[2]"),
        "tel":raw_data_each.xpath("//div[contains(@class,'col_half')]//li[3]/div/text()[1]"),
        "store_website":raw_data_each.xpath("//*[@id='main-menu']/li[10]/ul/li[3]/a/@href"),
        "city":okclinicname[2]
    },okclinicname[3]:{
        "address":raw_data_each.xpath("//div[contains(@class,'col_half')]//li[4]/div/text()[2]"),
        "tel":raw_data_each.xpath("//div[contains(@class,'col_half')]//li[4]/div/text()[1]"),
        "store_website":raw_data_each.xpath("//*[@id='main-menu']/li[10]/ul/li[4]/a/@href"),
        "city":okclinicname[3]
    }
  }
store_brand={
    "brand":"佳醫美人"
    }

# DICT整理成適合JSON

In [4]:
def fitdict():
# 刪除不需要的資訊
    temp[0]=temp[0].split("docsRawData.push(")
    del temp[0][0]
    temp[0]= ''.join(temp[0])
    temp[0]=temp[0].split(");\n\t")
    temp[0][-1]=temp[0][-1].split(');\n\njQuery.each(')
    del temp[0][-1][-1]
    return temp[0]

# JSON

In [5]:
def tojson():    
    fitdict()
    for x in range(len(temp[0])-1):
        temp[0][x]=ast.literal_eval(temp[0][x])
    temp[0][x+1]=ast.literal_eval(temp[0][-1][0])
    return temp

# intro取文字部分

In [6]:
def rectifyintro():
    tojson()
    for x in range(len(temp[0])):
        a=temp[0][x]['intro']
        soup=BeautifulSoup(a,'html.parser')
        temp[0][x]['intro']=soup.text
    return temp[0]

# 更改圖片格式

In [7]:
def changeimgform():
    rectifyintro()
    for x in range(len(temp[0])):
        temp[0][x]['featured_img']=(f"https://www.beautyclinic.com.tw{temp[0][x]['featured_img']}")
    return temp[0]

# DF

In [8]:
FILE_NAME = "佳醫美人"
today = time.strftime("%Y%m%d")
df = pd.DataFrame(changeimgform(),columns=['id','name','featured_img','locations','intro','job','sp','ename'])

# 每筆診所資訊(分隔)

In [9]:
def cleartext(x):
    if okclinicname[x] in df.loc[fir]['locations'][sec]['name']:
        df['address'][fir]=df.loc[fir]['address']+",".join(otherinfo[okclinicname[x]]['address'])+"  "
        df['city'][fir]=df.loc[fir]['city']+"".join(otherinfo[okclinicname[x]]['city'])+"  "
        df['tel'][fir]=df.loc[fir]['tel']+",".join(otherinfo[okclinicname[x]]['tel'])+"  "
        df['store_website'][fir]=df.loc[fir]['store_website']+",".join(otherinfo[okclinicname[x]]['store_website'])+"  "

# df增加診所資訊

In [10]:
df['address']=''
df['tel']=''
df['store_website']=''
df['city']=''
df['brand']=''
for fir in range(len(df['locations'])):
    for sec in range(len(df.loc[fir]['locations'])):
        for thi in range(4):
            cleartext(thi)
    df['brand'][fir]="".join(store_brand['brand'])

# CSV

In [11]:
df.rename(columns={'name':'doctor_name',"featured_img":"pic","sp":"specialty","intro":"experience"}, inplace=True)
df.drop(['id', 'locations','ename'], axis=1, inplace=True)
df = pd.DataFrame(df,columns=["brand","pic","doctor_name","store_name","city","address","tel","personal_website","store_website","specialty","description","case_study","experience","education"])
df.to_csv(f"{FILE_NAME}-{today}.csv",index=False)